In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate

In [ ]:
# Define the shape of your input images (height, width, channels)
input_shape = (64, 64, 3)

In [ ]:
# Create seven input layers
input1 = Input(shape=input_shape, name="Image1")
input2 = Input(shape=input_shape, name="Image2")
input3 = Input(shape=input_shape, name="Image3")
input4 = Input(shape=input_shape, name="Image4")
input5 = Input(shape=input_shape, name="Image5")
input6 = Input(shape=input_shape, name="Image6")
input7 = Input(shape=input_shape, name="Image7")

In [ ]:
fc1 = Dense(128, activation='relu')(input1)
fc2 = Dense(128, activation='relu')(input2)
fc3 = Dense(128, activation='relu')(input3)
fc4 = Dense(128, activation='relu')(input4)
fc5 = Dense(128, activation='relu')(input5)
fc6 = Dense(128, activation='relu')(input6)
fc7 = Dense(128, activation='relu')(input7)

In [ ]:
# Concatenate all the fully connected layers' outputs
concatenated = Concatenate()([fc1, fc2, fc3, fc4, fc5, fc6, fc7])

In [ ]:
# Additional Fully Connected layer on concatenated outputs
final_fc = Dense(256, activation='relu')(concatenated)

In [ ]:
model = Model(inputs=[text_input, metadata], outputs=output)


In [ ]:
# Compile the model 
model.compile(loss='categorical_crossentropy',optimizer='Adam', 
                           metrics=['accuracy'])

In [ ]:
print(model.summary())

# plot graph
display(plot_model(model, to_file="/content/drive/My Drive/thesis_sotiria/text_numerical.png"))

In [ ]:
model.fit(x= [X1_train_pad, scaled_X2_train], 
          y= y_train_ohe, validation_split=0.2, batch_size= 32,
          epochs= 100, callbacks = [early_stop, checkpoint])